In [2]:
# Install libraries
!pip install slack_bolt openai pinecone pyngrok

import os
from slack_bolt import App
from openai import OpenAI
from pinecone import Pinecone
from pyngrok import ngrok, conf
from google.colab import userdata

# Download and install ngrok binary with proper permissions
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok

# Set the ngrok path explicitly for pyngrok
ngrok_path = '/content/ngrok'
conf.get_default().ngrok_path = ngrok_path

# 1. Authenticate ngrok using our local binary
ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))

# 2. Initialize using Colab secrets
app = App(
    token=userdata.get('SLACK_BOT_TOKEN'),
    signing_secret=userdata.get('SLACK_SIGNING_SECRET')
)

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
index = pc.Index("gloria")

print("✅ All secrets loaded successfully!")
print("✅ Ngrok installed and authenticated!")
print(f"✅ Using ngrok binary at: {ngrok_path}")

Archive:  ngrok.zip
  inflating: ngrok                   
✅ All secrets loaded successfully!
✅ Ngrok installed and authenticated!
✅ Using ngrok binary at: /content/ngrok


In [3]:
# Helper to get embeddings
def embed(text):
    resp = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return resp.data[0].embedding

In [4]:
# Slash command handler
@app.command("/codesearch")
def handle_codesearch(ack, body, respond):
    ack()
    query = body["text"]

    if not query.strip():
        respond("Please provide a search query. Usage: `/codesearch <your query>`")
        return

    try:
        q_emb = embed(query)
        res = index.query(vector=q_emb, top_k=3, include_metadata=True)

        if not res.matches:
            respond(f"No code matches found for: `{query}`")
            return

        msg = f"*Top code matches for: '{query}'*\\n\\n"
        for i, match in enumerate(res.matches, 1):
            path = match.metadata.get("path", "Unknown path")
            snippet = match.metadata.get("snippet", "No snippet available")
            score = match.score
            msg += f"*{i}. {path}* (score: {score:.3f})\\n```{snippet}```\\n\\n"

        respond(msg)
    except Exception as e:
        respond(f"Error searching: {str(e)}")

In [ ]:
# STEP 4 - FIXED
import os
from pyngrok import ngrok, conf

# Set ngrok path to our downloaded binary
conf.get_default().ngrok_path = '/content/ngrok'

# Kill port 3000
!fuser -k 3000/tcp 2>/dev/null || true

# Connect ngrok
public_url = ngrok.connect(3000, bind_tls=True)
print(f"🎯 URL: {public_url}/slack/events")

# Start app
app.start(port=3000)

🎯 URL: NgrokTunnel: "https://sachiko-prodisarmament-undisastrously.ngrok-free.dev" -> "http://localhost:3000"/slack/events
⚡️ Bolt app is running! (development server)


Make.com simulation link
https://eu1.make.com/public/shared-scenario/PktkK8x4P4d/integration-webhooks